## 환경 맞추기

In [1]:
!wget https://www.python.org/ftp/python/3.7.0/Python-3.7.0.tgz
!tar xvfz Python-3.7.0.tgz
!Python-3.7.0/configure
!make
!sudo make install

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_calltips.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_codecontext.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_colorizer.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config_key.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_configdialog.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_debugger.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_delegator.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editmenu.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editor.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_grep.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help_a

In [2]:
!python --version

Python 3.7.0


In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661750 sha256=6b647a1ac525e6b1cb376cd8481ae97ac0f4c2b2b62cda69be6667e1ca04cada
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86

In [4]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-hs6x_yib/kobert-tokenizer_56e9d60b507a4a6aaecfa013f63d63e3
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-hs6x_yib/kobert-tokenizer_56e9d60b507a4a6aaecfa013f63d63e3
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=c1fa75e2298053728becd25673a8a682010985f671b02ba3fed8f1ae2572f182
  Stored in directory: /tmp/pip-ephem-wheel-cache-h3w41pnz/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


## 라이브러리 임포트

In [5]:
import pandas as pd
import numpy as np
# np.bool가 bool로 대체되었음을 명시
np.bool = bool

In [6]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from tensorflow.keras.preprocessing.text import Tokenizer

In [7]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

## 데이터 처리

In [9]:
data= pd.read_csv('/content/ChatbotData.csv')
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


### (사전)EDA

0: 일반, 1: 사랑, 2: 이별

In [10]:
data['label'].value_counts()

label
0    5290
1    3570
2    2963
Name: count, dtype: int64

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


결측치 x

### 데이터 전처리 및 로드

In [12]:
love_data= data[data['label'] != 0]
love_data

,Q,A,label
5290,1000일 만난 여자친구와 이별,더 오래 만날 사람 만날 거예요.,1
5291,10년 연애. 헤어졌습니다.,더 공허함이 크시겠네요.,1
5292,10년 연애사 되돌아보니 다 부질없네,더 좋은 사람 만나실 거예요.,1
5293,10년 연예의끝,더 마음이 허하겠어요.,1
5294,10년만나다 헤어지네,충분히 슬퍼하고 충분히 아파하다가 이겨내세요.,1
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [13]:
from sklearn.model_selection import train_test_split

x= list(love_data['Q'])
y= list(love_data['A'])
label= list(love_data['label'])

# x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state= 16)

### (사전)사용자 정의 사전 추가 및 저장

In [14]:
token= Tokenizer()
token.fit_on_texts(x)
x_token= [word for word, _ in token.word_index.items()]
x_token

['좋아하는',
 '너무',
 '썸',
 '이별',
 '사람',
 '내가',
 '사람이',
 '거',
 '어떻게',
 '같아',
 '헤어진지',
 '내',
 '것',
 '나',
 '싶어',
 '이제',
 '다시',
 '정말',
 '여자친구가',
 '수',
 '좀',
 '안',
 '남자친구가',
 '더',
 '잘',
 '오늘',
 '나를',
 '왜',
 '있을까',
 '자꾸',
 '또',
 '연애',
 '힘드네',
 '해',
 '있어',
 '될까',
 '타는',
 '있는데',
 '짝남이',
 '그',
 '하는',
 '연락',
 '짝사랑',
 '진짜',
 '사랑',
 '있는',
 '다',
 '날',
 '헤어진',
 '짝남',
 '후',
 '많이',
 '할까',
 '계속',
 '방법',
 '짝녀가',
 '다른',
 '이',
 '사랑이',
 '참',
 '모르겠어',
 '결국',
 '연락이',
 '없어',
 '사랑하는',
 '없는',
 '나한테',
 '마음이',
 '그냥',
 '전',
 '남자',
 '여자가',
 '여자친구',
 '남자가',
 '이젠',
 '힘들어',
 '같이',
 '결혼',
 '짝남한테',
 '헤어지고',
 '아직도',
 '먼저',
 '이렇게',
 '하고',
 '돼',
 '혼자',
 '여자',
 '어떤',
 '어제',
 '알았는데',
 '한',
 '줄',
 '못',
 '좋아',
 '이별을',
 '무슨',
 '때',
 '같은데',
 '안돼',
 '마음',
 '나는',
 '보고',
 '좋은',
 '했는데',
 '한달',
 '여친',
 '좋을까',
 '사람이랑',
 '이별한지',
 '싶다',
 '카톡',
 '나만',
 '해도',
 '갑자기',
 '마지막',
 '좋겠어',
 '사람을',
 '사람한테',
 '짝녀',
 '시간이',
 '지금',
 '만나고',
 '힘든',
 '걸까',
 '남자친구',
 '싶은데',
 '헤어졌습니다',
 '같은',
 '어떡해',
 '여친이',
 '이별후',
 '제가',
 '때문에',
 '합니다',
 '말',
 '오늘도',
 '걸',

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

# 사전학습 토크나이저와 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')#, return_dict=False) # 튜플 반환

# 사용자 정의 토크나이저 사용
## 사용자 정의 단어, 토크나이저에 추가
tokenizer.add_tokens(x_token)
## 모델의 임베딩 층 확장
model.resize_token_embeddings(len(tokenizer))

model.to(device)

# 추가된 상태의 모델과 토크나이저 저장
tokenizer.save_pretrained('./tokenizer_with_custom_vocab')
model.save_pretrained('./model_with_custom_vocab')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


### (사전) 질문을 임베딩 벡터로 변환하여 저장

In [ ]:
# 저장된 모델 및 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained('./tokenizer_with_custom_vocab')
model = BertModel.from_pretrained('./model_with_custom_vocab')
model.to(device)
model.eval()

# 질문을 임베딩 벡터로 변환하는 함수
def embed_question(question):
    with torch.no_grad():
        encoding = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=64,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        cls_embedding = last_hidden_state[:, 0, :].cpu().numpy()
        return cls_embedding

# 모든 질문을 임베딩 벡터로 변환하여 저장
embedded_questions = np.vstack([embed_question(q) for q in x])
np.save('embedded_questions_kobert.npy', embedded_questions)


## 모델 파인 튜닝

In [16]:
"""
class QADataset(Dataset):
    def __init__(self, questions, answers, tokenizer, max_len):
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]

        # 질문 및 답변을 토큰화 및 전처리
        encoding = self.tokenizer.encode_plus(
            question,
            text_pair= answer,
            add_special_tokens=True,
            max_length=self.max_len,
            # return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        # 전처리된 데이터 반환
        return {
            'input_ids': encoding['input_ids'].view(-1),
            'attention_mask': encoding['attention_mask'].view(-1),
            # 'labels': encoding['input_ids'].flatten(), # seq2seq업무를 위함
        }
"""

In [17]:
max_len = 64
batch_size = 64
num_epochs = 5
learning_rate =  5e-5

In [18]:
from torch.utils.data import DataLoader

# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# token = tokenizer.tokenize
data_train = QADataset(x, y, tokenizer, max_len)

# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = DataLoader(data_train, batch_size = batch_size, num_workers = 2)  # cpu count = 2

optimizer와 schedule 설정

In [ ]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# 옵티마이저 및 스케줄러 설정
# AdamW 는 Adam 옵티마이저의 변형으로, L2 정규화 (weight decay)를 더 잘 구현함
optimizer = AdamW(model.parameters(), lr= learning_rate)
total_steps = len(train_dataloader) * num_epochs  # 배치 수 * 에포크 수
# 학습 초기에 학습률을 천천히 증가(warmup)시키고 이후에는 선형적으로 감소 > 안정적 학습을 위해
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps * 0.1), num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from torch.nn.functional import cosine_similarity
from torch.nn.utils import clip_grad_norm_

# 학습 루프
model.to(device)
model.train()

for epoch in range(num_epochs):
    epoch_loss= 0
    for batch in train_dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state= outputs.last_hidden_state

        # 코사인 유사도 계산
        cos_sim= cosine_similarity(last_hidden_state[:, 0, :], last_hidden_state[:, 1, :])
        loss= 1 - cos_sim.mean()

        loss.backward()

        clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs} completed. Loss: {epoch_loss / len(train_dataloader)}")

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/5 completed. Loss: 0.03343352299292111
Epoch 2/5 completed. Loss: 0.0013069360001573284
Epoch 3/5 completed. Loss: 0.0008653934719493088
Epoch 4/5 completed. Loss: 0.0007001045838143061
Epoch 5/5 completed. Loss: 0.0006366794549145745


In [ ]:
# 모델 저장
torch.save(model.state_dict(), 'model.pth')

In [ ]:
# 질문에 대한 응답 생성 함수
def get_response(question, model, tokenizer, max_len):
    model.eval()
    with torch.no_grad():
        encoding = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        return last_hidden_state.cpu().numpy()

In [ ]:
# 예시 질문에 대한 응답 생성
question = "연애 고민이 있어요"
response = get_response(question, model, tokenizer, max_len)
print(response)

[[[ 0.01390528  0.01303202  0.06411833 ... -0.01632296 -0.0916929
    0.06495484]
  [ 0.01472737  0.01168557  0.06451527 ... -0.01600198 -0.09181871
    0.06472722]
  [ 0.015371    0.01321049  0.06399427 ... -0.01560237 -0.09197576
    0.06562712]
  ...
  [ 0.0136602   0.01336958  0.0634004  ... -0.01581043 -0.09049875
    0.0654232 ]
  [ 0.01366229  0.01336842  0.06340727 ... -0.01581672 -0.09050184
    0.06541782]
  [ 0.01366462  0.01337132  0.06341007 ... -0.01581906 -0.0905064
    0.06541616]]]


---

## 모델 테스트

In [30]:
import torch
from transformers import BertTokenizer, BertModel
from kobert_tokenizer import KoBERTTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 모델 및 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')
model.load_state_dict(torch.load('/content/drive/MyDrive/groom/model.pth'))
model.to(device)

# 평가모드로 전환
model.eval()

# 데이터 로드
x = list(love_data['Q'])
y = list(love_data['A'])

# 질문을 임베딩 벡터로 변환하는 함수
def embed_question(question):
    with torch.no_grad():
        encoding = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=64,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        cls_embedding = last_hidden_state[:, 0, :].cpu().numpy()
        return cls_embedding

# 기존 질문들을 임베딩 벡터로 변환하여 저장
embedded_questions = np.vstack([embed_question(q) for q in x])

# 사용자 질문에 가장 유사한 질문을 찾아 답변을 반환하는 함수
def find_best_answer(user_question):
    user_embedding = embed_question(user_question)
    similarities = cosine_similarity(user_embedding, embedded_questions)
    best_match_idx = np.argmax(similarities)
    return y[best_match_idx]

# 챗봇 동작
print("저는 당신의 연애/사랑/이별에 대한 고민에 대해 하나의 답을 내려줄 챗봇 상담가 '러브레터'입니다 🙂✉️🩷")

while True:
    user_question = input("\n요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)\n   >>  ")
    if user_question.lower() == '종료':
        print("행운을 빌어요🍀")
        break
    answer = find_best_answer(user_question)
    print(answer)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


RuntimeError: Error(s) in loading state_dict for BertModel:
	size mismatch for embeddings.word_embeddings.weight: copying a param with shape torch.Size([16603, 768]) from checkpoint, the shape in current model is torch.Size([8002, 768]).

## 유사도

In [23]:
import torch
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# KoBERT 모델과 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1').to(device)

def get_embedding(text):
    # pytorch tensor 형식으로 반환
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length= max_len)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# 각 질문에 대한 임베딩 생성
question_embeddings = [get_embedding(q) for q in x]

def find_best_answer(user_input):
    user_embedding = get_embedding(user_input)
    # 코사인 유사도 방식을 통해 유사 질문 데이터 찾기
    similarities = [cosine_similarity(user_embedding, qe)[0][0] for qe in question_embeddings]
    # 가장 유사도가 높은 질문을 찾아 그에 대응하는 답변 뱉기
    max_similarity = max(similarities)
    print('max_similarity', max_similarity)
    if max_similarity > 0.6:  # 임계값을 통해 무관한 질문 필터링
        best_match_index = similarities.index(max_similarity)
        return y[best_match_index]
    else:
        return "저는 연애 관련 상담만을 주로 하고 있어요. 다른 질문 부탁해요!"




The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [24]:
# 사용
print("저는 당신의 연애/사랑/이별 에 대한 고민에 대해 하나의 답을 내려줄 챗봇 상담가 '러브레터'입니다 🙂✉️🩷")
while True:
    user_question = input("\n요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)\n   >>  ")
    if user_question == '종료':
        print("행운을 빌어요🍀")
        break
    print(find_best_answer(user_question))

저는 당신의 연애/사랑/이별 에 대한 고민에 대해 하나의 답을 내려줄 챗봇 상담가 '러브레터'입니다 🙂✉️🩷

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  남자친구와 헤어졌어
max_similarity 0.8802399
더 허전할 거 같아요.

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  너무 보고싶어
max_similarity 0.8855055
얼른 만나러가세요.

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  내가 그 사람에게 연락해도 될까?
max_similarity 0.90918106
의지해도 좋을 사람이라면 의지해도 돼요.

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  그 사람이 괴로워 할 것 같아
max_similarity 0.8614672
좀 더 호감을 표현해보세요.

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  전 애인과 다시 만날 수 있을까
max_similarity 0.85736454
충분한 대화를 나눠보는게 좋을 것 같아요.

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  전 애인에게 다시 연락해볼까?
max_similarity 0.8708257
주말 전에 연락해서 주말에 만나야죠.

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  헤어진 사람에게 다시 연락하는거 어떻게 생각해?
max_similarity 0.8824679
위로를 해줘보세요.

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  헤어진 사람에게 다시 연락해도 될까
max_similarity 0.8966009
상대방이 부담스럽지 않을까요?

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  헤어진 관계는 끝인걸까
max_similarity 0.8933493
그게 최선이었을 거예요

In [26]:
import torch
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# KoBERT 모델과 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
modell = BertModel.from_pretrained('skt/kobert-base-v1').to(device)
modell.load_state_dict(torch.load('/content/drive/MyDrive/groom/model.pth'))

def get_embedding(text):
    # pytorch tensor 형식으로 반환
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=max_len)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# 각 질문에 대한 임베딩 생성
question_embeddings = [get_embedding(q) for q in x]

def find_best_answer(user_input):
    user_embedding = get_embedding(user_input)
    # 코사인 유사도 방식을 통해 유사 질문 데이터 찾기
    similarities = [cosine_similarity(user_embedding, qe)[0][0] for qe in question_embeddings]
    # 가장 유사도가 높은 질문을 찾아 그에 대응하는 답변 뱉기
    max_similarity = max(similarities)
    print('max_similarity', max_similarity)
    if max_similarity > 0.6:  # 임계값을 통해 무관한 질문 필터링
        best_match_index = similarities.index(max_similarity)
        return y[best_match_index]
    else:
        return "저는 연애 관련 상담만을 주로 하고 있어요. 다른 질문 부탁해요!"




The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


RuntimeError: Error(s) in loading state_dict for BertModel:
	size mismatch for embeddings.word_embeddings.weight: copying a param with shape torch.Size([16603, 768]) from checkpoint, the shape in current model is torch.Size([8002, 768]).

In [29]:
modell = BertModel.from_pretrained('skt/kobert-base-v1').to(device)
modell

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
# 사용
print("저는 당신의 연애/사랑/이별 에 대한 고민에 대해 하나의 답을 내려줄 챗봇 상담가 '러브레터'입니다 🙂✉️🩷")
while True:
    user_question = input("\n요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)\n   >>  ")
    if user_question == '종료':
        print("행운을 빌어요🍀")
        break
    print(find_best_answer(user_question))

---

In [ ]:
import gradio as gr

# 챗봇에 채팅이 입력되면 이 함수를 호출합니다.
# message는 유저의 채팅 메시지, history는 채팅 기록, additional_input_info는 additional_inputs안 블록의 정보를 받습니다.
def response(message, history, additional_input_info):
    # additional_input_info의 텍스트를 챗봇의 대답 뒤에 추가합니다.
    return "챗봇을 미완성하였습니다 " + additional_input_info

gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="말걸어주세요..", container=False, scale=7),
        title="어떤 챗봇을 원하심미까?",
        description="물어보면 답하는 챗봇임미다.",
        theme="soft",
        examples=[["안뇽"], ["요즘 덥다 ㅠㅠ"], ["점심메뉴 추천바람, 짜장 짬뽕 택 1"]],
        retry_btn="다시보내기 ↩",
        undo_btn="이전챗 삭제 ❌",
        clear_btn="전챗 삭제 💫",
        additional_inputs=[
            gr.Textbox("!!!", label="끝말잇기")
        ]
).launch()

## references

https://velog.io/@sseq007/Kobert-%EB%AA%A8%EB%8D%B8-%EC%82%AC%EC%9A%A91

https://velog.io/@sseq007/Kobert-%EB%AA%A8%EB%8D%B8-%EC%82%AC%EC%9A%A92

https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab

https://complexoftaste.tistory.com/2

공식 : https://github.com/SKTBrain/KoBERT?tab=readme-ov-file#korean-bert-pre-trained-cased-kobert